In [150]:
from time import process_time_ns
from types import new_class
from typing import List
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import seaborn as sns
np.set_printoptions(suppress=True)
movies = pd.read_csv('movies.csv', low_memory=False)
ratings = pd.read_csv('ratings.csv', low_memory=False)
asc = ratings.sort_values(by=['userId', 'timestamp'])
# print(asc)
# print(asc.values)
def getListUid(uid):
    ratings_temp = ratings
    movies_temp = movies
    
    final_dataset = ratings_temp.pivot(index='movieId',columns='userId',values='rating')
    # time_dataset = ratings_temp.pivot(index='movieId',columns='userId',values='timestamp')
    final_dataset = final_dataset.fillna(final_dataset.mean(axis=0))
    # print(time_dataset.head(10))
    desc = ratings_temp[ratings_temp['userId'] == uid].sort_values(by='timestamp', ascending=False)
    arr =desc.head(5).values
    # print(arr)
    movie_list = arr[:,1]
    # movie_list = np.append(movie_list,16)
    # print(movie_list)
    # print(ratings_temp)
    # a= ratings_temp.loc[movie_list]
    # print(a)
    # print(final_dataset)
    # Mean = ratings.groupby('movieId')['rating'].mean()
    # print(final_movie)
    final_dataset.fillna(0,inplace=True)
    # time_dataset.fillna(0,inplace=True)
    # time_dataset.fillna(0,inplace=True)
    # print(final_dataset.head())
    no_user_voted = ratings_temp.groupby('movieId')['rating'].agg('count')
    # print(no_user_voted)
    no_movies_voted = ratings_temp.groupby('userId')['rating'].agg('count')
    final_dataset = final_dataset.loc[no_user_voted[no_user_voted > 10].index,:]
    final_dataset = final_dataset.loc[:,no_movies_voted[no_movies_voted > 50].index]
    # print(final_dataset)
    final_dataset = final_dataset.loc[movie_list]

    final_dataset = final_dataset.T
    # print(final_dataset.iat[56])
    new_arr=  np.array(final_dataset)
    list_data = new_arr.tolist()
    knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)
    knn.fit(list_data)
    distances , indices =knn.kneighbors([list_data[0]],n_neighbors=10)
    rec_movie_indices = sorted(list(zip(indices.squeeze().tolist(),distances.squeeze().tolist())),key=lambda x: x[1])[:0:-1]
    # # print(rec_movie_indices)
    list_uid = list(zip(*rec_movie_indices))[0]
    return sorted(zip(distances.tolist()[0], indices.tolist()[0]))
    # return indices.tolist()[0]


In [151]:
def get_recommendation(uid,movie):
    # list_uid = getListUid(uid)
    # list_uid.pop(0)
    # clean_uid = []
    # for i in list_uid:
    #     clean_uid.append(i[1])
    # # print(clean_uid)
    # ratings_temp = ratings
    # movies_temp = movies
    
    # final_dataset = ratings_temp.pivot(index='movieId',columns='userId',values='rating')
    # time_dataset = ratings_temp.pivot(index='movieId',columns='userId',values='timestamp')
    # # final_dataset = final_dataset.fillna(final_dataset.mean(axis=0))
    # final_dataset.fillna(0,inplace=True)

    # final_dataset = final_dataset.T.loc[clean_uid].T
    # # print(final_dataset)
    # print(final_dataset)
    # # print()
    # # print(count_phim)
    
    # # print(test_id.iloc[1].at['rating'])
    # # for i in test_id.rows:
    # #     print(i['movieId'])
    # count_phim = []
    # np.set_printoptions(suppress=True,\
    # formatter={'float_kind':'{:0.0f}'.format})
    # for i in list_uid:
    #     test_id = final_dataset[final_dataset['userId'] == i[1]]
        
    #     x = int(test_id.size/4)
    #     for j in range(x):
    #         # print(int(test_id.iloc[j].at['rating']))
    #         # if (int(test_id.iloc[j].at['rating']) >= 4):
    #         count_phim.append([test_id.iloc[j].at['movieId'],test_id.iloc[j].at['rating']])
    
    # print(count_phim)
    # # unique, counts = np.unique(np.array(count_phim), return_counts=True)
    # # final_arr = np.asarray((unique, counts)).T.tolist()
    # # result = sorted(final_arr,key=lambda l:l[1], reverse=True)
    # # print (result)
    # # A + 
    list_uid = getListUid(uid)
    list_uid.pop(0)
    ratings_temp = ratings
    movies_temp = movies
    
    final_dataset = ratings_temp.pivot(index='movieId',columns='userId',values='rating')
    time_dataset = ratings_temp.pivot(index='movieId',columns='userId',values='timestamp')
    test_id = ratings_temp[ratings_temp['userId'] == 56]
    # print(test_id)
    # print()
    x = int(test_id.size/4)
    count_phim = []
    for i in range(x):
        if (test_id.iloc[i].at['rating'] >= 4):
            count_phim.append(test_id.iloc[i].at['movieId'])
    # print(count_phim)
    
    # print(test_id.iloc[1].at['rating'])
    # for i in test_id.rows:
    #     print(i['movieId'])
    count_phim = []
    # total = []
    np.set_printoptions(suppress=True,\
    formatter={'float_kind':'{:0.0f}'.format})
    for i in list_uid:
        # currId = ratings_temp[ratings_temp['userId'] == i[1]]
        # print(i[1])
        test_id = ratings_temp[ratings_temp['userId'] == i[1]]
        # print(test_id)
        # print()
        x = int(test_id.size/4)
        for j in range(x):
            if (test_id.iloc[j].at['rating'] >= 4):
                count_phim.append(test_id.iloc[j].at['movieId'])
    
    unique, counts = np.unique(np.array(count_phim), return_counts=True)
    final_arr = np.asarray((unique, counts)).T.tolist()
    top_recommendation = sorted(final_arr,key=lambda l:l[1], reverse=True)
    # print (top_recommendation)
    list_phim = []
    for i in range(10):
        list_phim.append(top_recommendation[i][0])
    
    # print(list_phim)
    # print(movies_temp)
    list_movie_name = []
    for i in list_phim:
        list_movie_name.append(movies.loc[i]['title'])
    return list_movie_name
    

print(get_recommendation(43,"Iron Man"))

       movieId                                title  \
0            1                     Toy Story (1995)   
1            2                       Jumanji (1995)   
2            3              Grumpier Old Men (1995)   
3            4             Waiting to Exhale (1995)   
4            5   Father of the Bride Part II (1995)   
...        ...                                  ...   
10324   146684        Cosmic Scrat-tastrophe (2015)   
10325   146878           Le Grand Restaurant (1966)   
10326   148238       A Very Murray Christmas (2015)   
10327   148626                 The Big Short (2015)   
10328   149532  Marco Polo: One Hundred Eyes (2015)   

                                            genres  
0      Adventure|Animation|Children|Comedy|Fantasy  
1                       Adventure|Children|Fantasy  
2                                   Comedy|Romance  
3                             Comedy|Drama|Romance  
4                                           Comedy  
...                  